<a href="https://colab.research.google.com/github/InsightofSPb/Automatic-word-processing-and-processing-of-images/blob/main/Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [ ]:
DATA_URL = "http://az.lib.ru/l/leskow_n_s/text_0246.shtml"

Устанавливаем библиотеки

In [ ]:
import warnings
warnings.filterwarnings('ignore')
!pip -q install rnnmorph

Создаём объект морфологического анализатора `RNNMorph`

In [ ]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [ ]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [ ]:
raw_text

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")


In [ ]:
# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [ ]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Лесков Николай Семенович. Левша\n\n\n\nЛесков Николай Семенович\r\nЛевша\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n[Помощь]\r\n\r\n'

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"A total of 466 'sentences'"

In [ ]:
tokenized_sentences

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [ ]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent] 
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться



Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [ ]:
len(predictions)

In [ ]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
y = list(filter(str.isalpha, non_uniq_tokens))



In [ ]:
len(y)

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам. 

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75. 

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [ ]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что']

In [ ]:
fdist = FreqDist()
lenght = 150
for word in y:
    fdist[word.lower()] += 1


In [ ]:
List = fdist.most_common()[0:lenght]
List

In [ ]:
count = 0
for elem in List:
  if elem[0] in STOPWORDS:
    count += 1
(lenght - count) / lenght

0.5333333333333333

Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

In [ ]:
u = list(filter(lambda x: x[1] > 10, List))
len(u)

126

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)
